<a href="https://colab.research.google.com/github/tak-creator/giganci/blob/main/naiwny_algorytm_Bayesa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import json
import pandas as pd

df = pd.read_json('tagi_gier_wspoldzielone.json')
df.head()

with open('tagi_gier_wspoldzielone.json') as f:
    data = json.load(f)

print(data)

[{'label': 'minecraft', 'tags': ['rankingi', 'sandbox', 'pistolet', 'event', 'smok', 'diamenty', 'kolekcje', 'klocki']}, {'label': 'fortnite', 'tags': ['tryb solo', 'snajper', 'online', 'autobus bojowy', 'misje', 'questy', 'lama', 'fortyfikacje']}, {'label': 'fortnite', 'tags': ['sezon', 'kopanie', 'misje', 'drużyna', 'fortyfikacje', 'biomy', 'v-bucksy', 'lama']}, {'label': 'roblox', 'tags': ['czat', 'karabin', 'tryb kreatywny', 'klocki', 'oddziały', 'studio', 'misje', 'brokhaven']}, {'label': 'lol', 'tags': ['przetrwanie', 'snajper', 'zombie', 'esport', 'skiny', 'budowanie', 'ulti', 'dżungla']}, {'label': 'fortnite', 'tags': ['roleplay', 'online', 'slurp', 'lama', 'pistolet', 'symulator', 'tryb kreatywny', 'sandbox']}, {'label': 'fortnite', 'tags': ['kolekcje', 'karabin', 'autobus bojowy', 'biomy', 'lama', 'strzelanie', 'loot', 'symulator']}, {'label': 'roblox', 'tags': ['tryb', 'strzał w głowę', 'oddziały', 'questy', 'studio', 'duety', 'brokhaven', 'symulator']}, {'label': 'lol', 'ta

In [8]:
df.head()

,label,tags
0,minecraft,"[rankingi, sandbox, pistolet, event, smok, dia..."
1,fortnite,"[tryb solo, snajper, online, autobus bojowy, m..."
2,fortnite,"[sezon, kopanie, misje, drużyna, fortyfikacje,..."
3,roblox,"[czat, karabin, tryb kreatywny, klocki, oddzia..."
4,lol,"[przetrwanie, snajper, zombie, esport, skiny, ..."


In [9]:
'''
{
    "label": "minecraft"
    "tags": [
        "rankingi",
        "sandbox",
        "event",
        ...
    ]
}
'''


'\n{\n    "label": "minecraft"\n    "tags": [\n        "rankingi",\n        "sandbox",\n        "event",\n        ...\n    ]\n}\n'

In [10]:
count = {}

for game in data:
    label = game["label"]
    if label not in count:
        count[label] = 0
    count[label] += 1

print(len(data))
print(len(count))

300
5


In [11]:
for label in count:
    print(label, count[label])

minecraft 60
fortnite 60
roblox 60
lol 60
cs 60


In [12]:
tag_count = {}
for game in data:
    tags = game["tags"]
    for tag in tags:
        if tag not in tag_count:
            tag_count[tag] = 0
        tag_count[tag] += 1

print(len(tag_count))

74


In [29]:
import math
import random

def load_path(path):
    with open(path, encoding="utf-8") as f:
        data = json.load(f)
    return data

def train_test_split(data, test_ratio = 0.2):
    random.shuffle(data)
    cut = int(len(data) * (1-test_ratio))

    print(f"Ilość próbek zbioru treningowego: {len(data[:cut])}")
    print(f"Ilość próbek zbioru testowego: {len(data[cut:])}")
    return data[cut:], data[:cut]

def build_vocabulary(train):
    vocab = set()
    for rec in train:
        vocab.update(rec["tags"])
    print(vocab)
    return vocab

def train_naive_bayes(train, vocab, alpha = 0.1):
    '''Klasyfikacja gier na podstawie tagów'''
    class_count = {}
    word_counts = {}
    total_words = {}
    for rec in train:
        c = rec["label"]
        class_count[c] = class_count.get(c, 0) + 1

        word_counts.setdefault(c, {}) # wartość startowa pusty słownik
        total_words.setdefault(c, 0) # wartość początkowa na 0

        for tag in rec["tags"]:
            word_counts[c][tag] = word_counts[c].get(tag, 0) + 1
            total_words[c] += 1

        model = {
            "class_counts": class_count,
            "words_counts": word_counts,
            "total_words": total_words,
            "vocab": vocab,
            "alpha": alpha,
            "total_docs": len(train)
        }
        print(model)
        return model

    def log_prob(model, rec, class_name):
        #podstawowe pradopodobieństwo
        log_prob = math.log(model["class_counts"][class_name] / model["total_docs"])
        V = len(model["vocab"])
        a = model["alpha"]
        #prawdopodobieństwo warunkowe tag|klasa
        for tag in rec["tags"]:
            word_count = model["words_counts"][class_name].get(tag, 0)
            log_prob += math.log((word_count + a) / (model["total_words"][class_name] + a * V))
        return log_prob

    def predict(model, rec):
        best_class = None
        best_log = None
        for c in model["class_counts"]:
            log_p = log_prob(model, rec, c)
            if lp > best_log:
                best_log = lp
                best_class = c
        return best_class

In [30]:
def main():
    path="tagi_gier_wspoldzielone.json"
    data = load_path(path)
    train, test = train_test_split(data, test_ratio = 0.2)
    vocab = build_vocabulary(train)
    model = train_naive_bayes(train, vocab, alpha= 1.0)

In [31]:
main()

Ilość próbek zbioru treningowego: 240
Ilość próbek zbioru testowego: 60
{'prywatny serwer', 'snajper', 'kopanie', 'sklep', 'brokhaven', 'obby', 'ulti', 'autobus bojowy', 'przetrwanie', 'budowanie', 'emotki', 'misje', 'smok', 'multiplayer', 'event', 'tickrate', 'dust2', 'lama', 'crafting', 'nether', 'czat', 'drużyna', 'pistolet', 'wyzwania', 'tryb solo', 'runy', 'questy', 'strzelanie', 'kolekcje', 'flash', 'loot', 'duety', 'strzał w głowę', 'esport', 'roleplay', 'online', 'battle', 'nexus', 'klocki', 'tryb', 'v-bucksy', 'waluta', 'redstone', 'symulator', 'bohaterowie', 'rankedy', 'fortyfikacje', 'burza', 'aktualizacja', 'zombie', 'hardcore', 'dżungla', 'tryb kreatywny', 'karabin', 'skiny', 'baron', 'sezon', 'parkour', 'awp', 'rankingi', 'slurp', 'bomba', 'diamenty', 'robuxy', 'survival', 'oddziały', 'creepery', 'biomy', 'sandbox', 'mapa', 'studio', 'lua', 'wydarzenia', 'arena'}
{'class_counts': {'cs': 1}, 'words_counts': {'cs': {'multiplayer': 1, 'crafting': 1, 'tryb solo': 1, 'hardcore